Password manager program which takes master password for encryption from user and then encrypts the file. The website names are taken as input from the user and the passwords are generated using the random function.

In [14]:
#Please Run this code of block for the first time of execution of the code inorder to create the needed files.
#If the files are already exist please donot execute this block
website_name_in_file = open("site_name.dat","w")#To store website names
password_name_file = open("passwords.dat","w")#to set the encrypted passwords
salt_file = open("salt.dat","w")#To save the associated random salts
tag_file = open("tag.dat","w")#To save the tags
nonce_file = open("nonce.dat","w")#To save the nonce value


In [15]:
#This block of code is used to generate the 14 character password for the input website of the user

import random #library to generate random numbers
import string#library to generate strings

def generate_14character_password():
    special_characters=['!@#']#Given condition is to include only these special characters in our program
    #To generate a random array with lower, upper and digits
    random_password_array = string.ascii_lowercase + string.ascii_uppercase + string.digits#Used to generate the random string using all the lower,upper and digits
    
    random_password = random.choice(string.ascii_lowercase)#Select one lower case letter 
    random_password += random.choice(string.ascii_uppercase)#Select one upper case letter and append to arrray
    random_password += random.choice(string.digits)#Select a random digit and append to array
    random_password += random.choice(special_characters)#get the special characters in random order and append to array

    for i in range(8):
      random_password +=random.choice(random_password_array)#generate remainig password and append to string

    rlist = list(random_password)#convert to list for further randomization
    random.SystemRandom().shuffle(rlist)#to shuffel the elements in array
    random_password_final = ''.join(rlist)
    return random_password_final#returning the final password for the website






In [16]:
#This block of code is mainly for the purpose of the encryption and decryption of the encrypted passwords with the help of the master password.
from base64 import b64encode, b64decode
import hashlib
from Cryptodome.Cipher import AES
import os
from Cryptodome.Random import get_random_bytes#using the crptodome library to work with encryption and decryption

def encrypt(website_password, password):
    # the below is to genreate a random salt key block
    salt = get_random_bytes(AES.block_size)
    
    # now we generate the private key from the master password that we have given for the purpose of the encryption of the website password to be stored in the file.
    private_key = hashlib.scrypt(
        password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)
    #Now we need to create a cipher config to encrypt using the private key
    
    cipher_config = AES.new(private_key, AES.MODE_GCM)

    # return a dictionary with the encrypted text
    cipher_text, tag = cipher_config.encrypt_and_digest(bytes(website_password, 'utf-8'))
    
    cipher_text_1 = b64encode(cipher_text).decode('utf-8')#saving the encrypted password in a temporary variable
    salt1 = b64encode(salt).decode('utf-8')#saving the salt in temp variable
    nonce1 = b64encode(cipher_config.nonce).decode('utf-8')#Saving the nonce in temp variable
    tag1 = b64encode(tag).decode('utf-8')#saving the tag
    
    #The below block of code represents the opening of the files to save the encrypted details in the files and then closing the files after the required operations are performed.
    password_name_file = open("passwords.dat","a")
    salt_file = open("salt.dat","a")
    tag_file = open("tag.dat","a")
    nonce_file = open("nonce.dat","a")
    
    password_name_file.write(cipher_text_1 + "\n")
    
    salt_file.write(salt1 + "\n")
    
    tag_file.write(tag1 + "\n")
    
    nonce_file.write(nonce1 + "\n")
    
    password_name_file.close()
    
    salt_file.close()
    
    tag_file.close()
    
    nonce_file.close()
    
    
    
    
    
    
        

def decrypt(password_to_decrypt,salt,tag,nonce,password):
    
    #The below 4 lines of code is for decoding the string format data to the byte streams to be used for the decoding purpose.
    cipher_text = b64decode(password_to_decrypt) 
    salt = b64decode(salt)
    tag = b64decode(tag)
    nonce = b64decode(nonce)
    
    

    #the key to decode the specific information is generated form the key and salt, tag, nonce by creating the cipher config.
    #The decrypt_and_verify is used to decrypt and very the data whether decryption is done properly or not.
    private_key = hashlib.scrypt(
        password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    
    cipher = AES.new(private_key, AES.MODE_GCM, nonce=nonce)

    # decrypt the cipher text
    decrypted = cipher.decrypt_and_verify(cipher_text, tag)

    return decrypted#Returing the decrypted value.


def master_password(local_website_password, password):
    
    encrypt(local_website_password, password)
    print("\nFiles Have been encrypted with the master password")
    
    


In [18]:
# The code for main menu and performing the operations.


# the exit commands are mostly written in the try catch block because they promote abrupt termiantion of the program
#to avoud the abouve problem and to achieve proper termination, we use the try catch to get the  proper termination of the code.
import sys#used ot work with system related functions
import os

mp = input("\nPlease Enter Your master password:  ")

def masterpasswordcreate():
    
    mp1 = input("Please confirm Your master password:")
    if(mp == mp1):
        flag=1
    else:
        print("Please Re-run the code to reenter the masterpassword")
        try:
            sys.exit()
        except:
            
            #print("\nProgram Terminated")
            flag=0
            return flag
    

q = masterpasswordcreate()#custom method used to create the master password and then get the flag which determies the program execuion 

flag=1
while(flag):

  
  if(q == 0):
        try:
          sys.exit()
        except :
          print("\nProgram Terminated")
          break
  
  print("\nPlease select the following operations:")
  print("\n1. Add a website")
  
  print("\n2.View the details of a website")
  print("\n3.Exit from program")
  option = input("\nPlease make a selection:")

 
  if(option == '3'):
    try:
      sys.exit()
    except :
      print("\nProgram Terminated")
      break
  elif(option == '1'):
    #master_password_first = mp
    master_password_actual = input("\nPlease Enter the master password to add the website:")
    if(master_password_actual != mp):#check whether the reentered master password is smae as the first entered master password
        try:
            sys.exit()#used to exit from the program execution
        except:
            print("\nInvalid master password. Exiting from the program")
            break
    website_name=input("\nPlease enter the website name:")#get the website name from user to save in the db or file
    website_name_in_file = open("site_name.dat","a")
    website_name_in_file.write(website_name + "\n")
    website_name_in_file.close()
    
    website_password = generate_14character_password()#calls the method which is used to generate the random password for the website with the requierd specifications
    
    master_password(website_password,master_password_actual)#used to encrypt the master password and then with that encrypt the website password and then the encrypted website password is stored in the "passwords.dat" file
    print("\nThe website details added are:",website_name)
    #print("\nThe password is :",website_password)
  
  elif(option =='2'):
    master_password_actuall = input("\nPlease Enter the master password to retreve the website:")
    if(master_password_actuall != mp):
        try:
            sys.exit()
        except:
            print("\nInvalid master password. Exiting from the program")
            break
    read_website_details = input("\nplease eneter the site to read details:")
    
    #website_name_in_file = open("site_name.dat","r")
    
    site_names_array = []
    site_names = open("site_name.dat", "r") #open the file in read mode
    for sites in site_names:
        site_names_array.append(sites.strip("\n"))#populate the array site_names_array with data form site_name.dat file
    site_names.close()#close the open file
    
    
    salt_names_array = []
    salt_names = open("salt.dat", "r")  #open file in read mode
    for salts in salt_names:
        salt_names_array.append(salts.strip("\n"))#populate the data in salt_names_array with data form salt.dat file
    salt_names.close()#close the open file
    
    pass_names_array = []
    pass_names = open("passwords.dat", "r") #open file in read mode
    for passw in pass_names:
        pass_names_array.append(passw.strip("\n"))#used to populate the pass_names_array with data from passwords.dat file
    pass_names.close()#close the open file
    
    
    tag_names_array = []
    tag_names = open("tag.dat", "r") #to open the file in read mode
    for tag in tag_names:
        tag_names_array.append(tag.strip("\n"))#used to populate the tag_names_array with data from the tad.dat file. 
    tag_names.close()#close the open file
    
    nonce_names_array = []
    nonce_names = open("nonce.dat", "r")  #opening file in read mode
    for nonce in nonce_names:
        nonce_names_array.append(nonce.strip("\n"))#this populates the nonce_names_array with the data present in nonce.dat file 
    nonce_names.close()#to close the open file
    
    
    
    
    
    
    
    x = True
    position = -1
    while(x):
        
        for i in range(len(site_names_array)):#finding the position where the actual website is there, so that we could extract the other necessary information corresponding to it.
            if (site_names_array[i] == read_website_details):
                position = i
                x = False
        x = False
    website_name_in_file.close()
    if (position != -1):
        #master_password_re = input("\nPlease Re-enter your master password:")
        
        
        
        if( mp== master_password_actual):#Checking whether the master passwords are same 
            
            decrypted = decrypt(pass_names_array[i],salt_names_array[i],tag_names_array[i],nonce_names_array[i],mp)
            #the above function is to get the decrypted password from the encrypted password for the website name.
            
            print("\nThe Website found in the database is : ", site_names_array[i])
            
            print("\nThe Actual password of the website is :", bytes.decode(decrypted))
        else:
            print("\n Invalid Master password")
            try:
              sys.exit()
            except :
              print("\nProgram Terminated")
              break
            
            
        
        
        
        
    else:
        print("\nThe Given website doesnot Exist")
    #The below is for the reinitializing of the arrays used to save the detials form the files to empty for the purpose of safer site to avoid buffer errors
    nonce_names_array = ""
    tag_names_array = ""
    pass_names_array = ""
    salt_names_array = ""
    site_names_array = ""
    
    
    
  else:
    os.system('clear')
    print("\nPlease enter a valid choice option")



    



Please Enter Your master password:  gp0227
Please confirm Your master password:gp0227

Please select the following operations:

1. Add a website

2.View the details of a website

3.Exit from program

Please make a selection:3

Program Terminated


**References**

https://www.geeksforgeeks.org/python-convert-string-to-bytes/

https://www.pythonforbeginners.com/files/with-statement-in-python

https://docs.python.org/3/library/hashlib.html

https://www.codegrepper.com/code-examples/python/open+dat+file+python

https://www.kite.com/python/answers/how-to-write-a-string-to-a-file-on-a-new-line-every-time-in-python

https://stackoverflow.com/questions/15718068/search-file-and-find-exact-match-and-print-line

https://stackoverflow.com/questions/2081836/how-to-read-specific-lines-from-a-file-by-line-number

https://www.kite.com/python/answers/how-to-read-specific-lines-of-a-text-file-in-python

https://www.geeksforgeeks.org/reading-writing-text-files-python/

https://realpython.com/read-write-files-python/

https://stackoverflow.com/questions/21839803/how-to-append-new-data-onto-a-new-line

https://thispointer.com/how-to-append-text-or-lines-to-a-file-in-python/

https://docs.python.org/3/library/sys.html

https://www.quora.com/How-can-I-create-Random-passwords-having-upper-and-lower-case-letters-a-special-character-and-numbers

https://www.codegrepper.com/code-examples/python/python+generate+random+password

https://qvault.io/cryptography/aes-256-cipher-python-cryptography-examples/

https://www.mssqltips.com/sqlservertip/5173/encrypting-passwords-for-use-with-python-and-sql-server/

https://pycryptodome.readthedocs.io/en/latest/

https://pycryptodome.readthedocs.io/en/latest/src/installation.html#windows-from-sources

https://pycryptodome.readthedocs.io/en/latest/src/examples.html#generate-public-key-and-private-key

https://pycryptodome.readthedocs.io/en/latest/src/examples.html#encrypt-data-with-aes

https://pycryptodome.readthedocs.io/en/latest/src/hash/sha256.html

https://www.brendanlong.com/python-encryption-example.html

https://www.sqlshack.com/encrypting-passwords-with-python-scripts-in-sql-notebooks-of-azure-data-studio/

https://asciinema.org/a/tEGTsXmEMJALLhuYljnRWf8Oh

https://pynative.com/python-generate-random-string/#h-create-random-password-with-special-characters-letters-and-digits

https://www.geeksforgeeks.org/python-string-digits/

https://stackoverflow.com/questions/19213232/python-v3-random-password-generator/19213271

https://stackoverflow.com/questions/12524994/encrypt-decrypt-using-pycrypto-aes-256

https://stackoverflow.com/questions/13899342/can-we-use-json-as-a-database

https://stackoverflow.com/questions/61378005/decrypting-aes-256-cbc-cipher-with-a-passcode-built-using-pbkdf2-in-python

https://asecuritysite.com/encryption/aes_gcm2

https://www.quickprogrammingtips.com/python/aes-256-encryption-and-decryption-in-python.html

https://www.programcreek.com/python/example/93236/pbkdf2.PBKDF2




